# Example Embeddings OpenSource

In [ ]:
!pip install -U sentence-transformers

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

/home/linuxlab/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# !pwd
df_avatar = pd.read_csv('../../Info_data/atla-episodes-scripts.csv')
df_avatar.head(10)

/home/linuxlab/cursos/templates/functions


,Character,script,ep_number,Book,total_number
0,NaN,"As the title card fades, the scene opens onto ...",1,1,1
1,Sokka,It's not getting away from me this time. [Clos...,1,1,1
2,NaN,"The shot pans quickly from Sokka to Katara, wh...",1,1,1
3,Katara,"[Happily surprised.] Sokka, look!",1,1,1
4,Sokka,"[Close-up of Sokka; whispering.] Sshh! Katara,...",1,1,1
5,NaN,"Behind Sokka, Katara is still making circular ...",1,1,1
6,Katara,[Struggling with the water that passes right i...,1,1,1
7,NaN,The bubble containing her fish slowly drifts a...,1,1,1
8,Katara,[Exclaims indignantly.] Hey!,1,1,1
9,NaN,"As Sokka lets out a gasp of discomfort, the fi...",1,1,1


In [13]:
# Convirtiendo a embeddings 2 frases:
sentences = ['este es el primer ejemplo','y este es el segundo ejemplo']
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
# tamaño:
embeddings.shape

(2, 384)

In [14]:
# Num max de embeddings que soporta el modelo
model.max_seq_length

256

In [15]:
# embeddings con GPU:
embeddings = model.encode(df_avatar['script'],batch_size=64, show_progress_bar=True)

Batches: 100%|██████████| 209/209 [00:12<00:00, 16.23it/s]


In [ ]:
# embeddings con CPU: (mil horas)
embeddings_cpu = model.encode(df_avatar['script'],batch_size=64, show_progress_bar=True, device='cpu')

In [19]:
# Guardando el dataset
df_avatar['embeddings'] = embeddings.tolist()
df_avatar

,Character,script,ep_number,Book,total_number,embeddings
0,NaN,"As the title card fades, the scene opens onto ...",1,1,1,"[-0.07994643598794937, 0.08622030913829803, 0...."
1,Sokka,It's not getting away from me this time. [Clos...,1,1,1,"[0.017791548743844032, 0.05984479933977127, -0..."
2,NaN,"The shot pans quickly from Sokka to Katara, wh...",1,1,1,"[-0.02218461222946644, 0.05684064328670502, -0..."
3,Katara,"[Happily surprised.] Sokka, look!",1,1,1,"[-0.018152210861444473, 0.11454901844263077, 0..."
4,Sokka,"[Close-up of Sokka; whispering.] Sshh! Katara,...",1,1,1,"[-0.013881666585803032, 0.08426917344331741, -..."
...,...,...,...,...,...,...
13364,Suki,And why did you paint me firebending?,21,3,61,"[-0.015251071192324162, 0.0721794068813324, -0..."
13365,Sokka,I thought it looked more exciting that way. [M...,21,3,61,"[0.044964782893657684, 0.020709633827209473, 0..."
13366,Iroh,"[Points at painting.] Hey, my belly's not that...",21,3,61,"[0.060673948377370834, 0.11253121495246887, 0...."
13367,Toph,Well I think you all look perfect! [They laugh.],21,3,61,"[-0.014031446538865566, 0.06187146157026291, 0..."


In [24]:
# Texto para buscar:
query_embedding = model.encode(['Teach me please'])

# agrega nueva columna de que tan similar es con lo que se esta buscando
# entre mas cerca a 1 mas se parece
df_avatar['similarity'] = df_avatar.embeddings.apply(lambda x : util.cos_sim(x,query_embedding[0]))
df_avatar.head()

,Character,script,ep_number,Book,total_number,embeddings,similarity
0,NaN,"As the title card fades, the scene opens onto ...",1,1,1,"[-0.07994643598794937, 0.08622030913829803, 0....",[[tensor(0.1533)]]
1,Sokka,It's not getting away from me this time. [Clos...,1,1,1,"[0.017791548743844032, 0.05984479933977127, -0...",[[tensor(0.1515)]]
2,NaN,"The shot pans quickly from Sokka to Katara, wh...",1,1,1,"[-0.02218461222946644, 0.05684064328670502, -0...",[[tensor(0.1335)]]
3,Katara,"[Happily surprised.] Sokka, look!",1,1,1,"[-0.018152210861444473, 0.11454901844263077, 0...",[[tensor(0.0443)]]
4,Sokka,"[Close-up of Sokka; whispering.] Sshh! Katara,...",1,1,1,"[-0.013881666585803032, 0.08426917344331741, -...",[[tensor(0.1546)]]


In [29]:
# Se ordena descendente para ver lo mas parecido de manera semantica:
df_avatar.sort_values('similarity',ascending=False).head(5)

,Character,script,ep_number,Book,total_number,embeddings,similarity
3674,Jeong Jeong,"Yes, yes, I will teach you.",16,1,16,"[0.010986867360770702, -0.01476271077990532, -...",[[tensor(0.5569)]]
12755,Zuko,Teaching you a lesson!,18,3,58,"[0.11967292428016663, 0.0042418488301336765, -...",[[tensor(0.5376)]]
4215,Katara,"But you still won't teach me, will you?",18,1,18,"[0.02912251278758049, -0.022517632693052292, -...",[[tensor(0.5253)]]
1595,Zuko,You taught me well.,7,1,7,"[0.05541105568408966, 0.07495422661304474, 0.0...",[[tensor(0.4829)]]
11502,Aang,Maybe I need a little more instruction. Perhap...,13,3,53,"[-0.016866054385900497, 0.007027492858469486, ...",[[tensor(0.4364)]]


In [30]:
# Se ordena ascendente para ver lo q menos se parece: 
df_avatar.sort_values('similarity',ascending=True).head(5)

,Character,script,ep_number,Book,total_number,embeddings,similarity
10344,Katara,"It's a wanted poster of you! ""The Runaway!"" Is...",7,3,47,"[-0.02448396012187004, 0.0721433088183403, -0....",[[tensor(-0.1342)]]
10964,Mechanist,"Yes, I was, and I think the Fire Nation will b...",10,3,50,"[0.017817316576838493, 0.028031155467033386, -...",[[tensor(-0.1309)]]
1582,Earthbender captain,I agree. He must be dealt with immediately and...,7,1,7,"[0.01889757066965103, 0.0808662623167038, -0.0...",[[tensor(-0.1252)]]
3201,Sokka,"Flowers are fine once you're married, but at t...",14,1,14,"[-0.02318212203681469, -0.051059335470199585, ...",[[tensor(-0.1195)]]
6144,Young Azula,Oh yes he is! He found out his son died and he...,7,2,27,"[-0.03853252902626991, -0.05967513844370842, 0...",[[tensor(-0.1152)]]
